# Notebook to visualize SWOT discharge data obtained from SoS

The SWORD of Science (SoS) is a community-driven dataset created as part of the cloud-based Confluence workflow, which facilitates rapid access to and processing of SWOT data. SoS serves as both an input and output repository for the workflow. It includes prior hydrologic data such as global agency gauge records, the Water Balance Model (WBM), and Global Reach-scale A priori Discharge Estimates (GRADES). Additionally, it stores the final results from each module of the Confluence workflow, providing key discharge parameters required for the SWOT mission.

<img src="Global-River-Discharge-from-SWOT.jpg" alt="SWOTDischarge" width="500"/>

<br>

Image Source: *Michael Durand, Colin J. Gleason, Tamlin M. Pavelsky, Renato Prata de Moraes Frasson, Michael Turmon, Cédric H. David, Elizabeth H. Altenau, Nikki Tebaldi, Kevin Larnier, Jerome Monnier, Pierre Olivier Malaterre, Hind Oubanas, George H. Allen, Brian Astifan, Craig Brinkerhoff, Paul D. Bates, David Bjerklie, Stephen Coss, Robert Dudley, Luciana Fenoglio, Pierre-André Garambois, Augusto Getirana, Peirong Lin, Steven A. Margulis, Pascal Matte, J. Toby Minear, Aggrey Muhebwa, Ming Pan, Daniel Peters, Ryan Riggs, Md Safat Sikder, Travis Simmons, Cassie Stuurman, Jay Taneja, Angelica Tarpanelli, Kerstin Schulze, Mohammad J. Tourian, Jida Wang. 2023. A Framework for Estimating Global River Discharge From the Surface Water and Ocean Topography Satellite Mission, *Water Resources Research*, 59(4). [https://doi.org/10.1029/2021WR031614](https://doi.org/10.1029/2021WR*


Key documents: https://podaac.jpl.nasa.gov/dataset/SWOT_L4_DAWG_SOS_DISCHARGE

This notebook was written by Anthony Castronova (acastronova@cuahsi.org) and Irene Garousi-Nejad (igarousi@cuahsi.org), CUAHSI.

## 1 Set up environment

The Python cells below need to be run each time the notebook is executed. The set up the needed libraries to run here in CUAHSI's Jupyter Hub cloud. You’ll need valid **Earthdata** login credentials to access the data used in this notebook. If you don’t already have an account, you can create one at https://urs.earthdata.nasa.gov/. **Without these credentials, you won't be able to access the required datasets or run the notebook as intended.**

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import xarray
import earthaccess
import h5netcdf
import getpass
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Prompt the user for Earthdata credentials interactively
username = input("Enter your Earthdata username: ")
password = getpass.getpass("Enter your Earthdata password (input hidden): ")

# Set environment variables
os.environ["EARTHDATA_USERNAME"] = username
os.environ["EARTHDATA_PASSWORD"] = password

# Login using the credentials
auth = earthaccess.login(strategy="environment")

## 2 Search for SWOT discharge data granules

The following code cell uses the Earthdata `earthaccess` API to search for SWOT discharge data by specifying a product name and a date range. It returns basic information about the available data files that match our search. 

In [ ]:
# Search and locate granules
granule_info = earthaccess.search_data(
    short_name="SWOT_L4_DAWG_SOS_DISCHARGE",
    temporal=("2023-04-07", "2023-04-26"),
)

The output contains a series of SWOT discharge data granules. Each granule corresponds to a specific version (Version:1 ) of the SWOT_L4_DAWG_SOS_DISCHARGE data collection. For each granule, the following metadata is shown:
* Spatial Coverage: The geographic bounding box where the data applies
* Temporal Coverage: The date and time range the granule covers.
* Size: The size of the data file.
* Data: URLs linking to NetCDF files—typically a `*_results.nc` file with output data and a `*_priors.nc` file with input or constraint data used in the model.

In [ ]:
# Display the metadata for the first two items
granule_info[0:2]

After obtaining granule_info, use the Earthdata `open` function to generate a list of cloud-hosted file-like objects, where each item represents a SWOT SoS NetCDF file. Each file is either a `*_results.nc` file containing model outputs or a `*_priors.nc` file with input parameters used in the modeling process. This step is essential before accessing and exploring specific data groups within the files, as it provides the necessary remote file references for lazy cloud-based access.

In [ ]:
%%time

files = earthaccess.open(granule_info)

In [ ]:
# Print the first two items of the list
files[0:2]

The items are ordered sequentially in the list. So files[0] is the first results file, files[1] is its associated priors file, and so on. The SoS dataset is organized into multiple named groups, each corresponding to a distinct module (algorithm) used in the Confluence workflow. Using Python's `h5netcdf` package, we can list these groups. You can explore detailed descriptions of these groups and their contents in the official data tutorial at https://podaac.github.io/tutorials/notebooks/datasets/SWOT_L4_DAWG_SOS_DISCHARGE.html.


In [ ]:
%%time

with h5netcdf.File(files[4], mode='r') as f:
    print(list(f.keys()))  

These groups can be accessed directly from the SoS NetCDF file using tools like xarray as shown in the next section. 

## 3 Access and explore SoS reach data using xarray 

We use xarray twice here to directly load two different groups from the same SWOT SoS cloud-hosted **results** files:

* `reaches` group: contains metadata about each river reach (such as reach_id, river_name, etc.).
* `hivdi` group: contains the discharge time series (Q) estimated by the HiVDI algorithm, using the same set of reaches.

You notice that both datasets share the same dimension size (`num_reaches`). We load the reaches and hivdi groups separately because both sets of information are needed for this analysis and visualization. 

In [ ]:
%%time

ds_reaches = xarray.open_dataset(files[4],
                                 group='reaches',
                                 engine='h5netcdf',
                                 decode_cf=False,    
                                 decode_times=False, 
                                 decode_coords=False)
ds_reaches

In [ ]:
%%time

ds_hivdi = xarray.open_dataset(files[4],
                           group='hivdi',
                           engine='h5netcdf',
                           decode_cf=False,    
                           decode_times=False, 
                           decode_coords=False,
                        )
ds_hivdi

The `reaches` group provides structural metadata for each reach, while the `hivdi` group contains the discharge time series estimates. We will combine them using the common `num_reaches` coordinate to create a unified dataset that links each reach's discharge values with its corresponding metadata, enabling more effective exploration and interpretation.

In [ ]:
# Merge two datasets using the common coordinate
ds_merged = xarray.combine_by_coords([ds_reaches, ds_hivdi])
ds_merged

## 4 Filter discharge data by river name


This section shows how to extract discharge data for a specific river from the merged SoS dataset. The dataset includes discharge time series linked to geographic coordinates that intersect with river reaches defined in the SWORD database. To focus our analysis, we isolate a subset of reaches associated with a user-defined river, for example the **Rhine River**, using a simple filtering approach.


In [ ]:
# Select a river of interest
RIVER_NAME = "Rhine"

Select all elements from the reaches dataset that are associated with the river named above.

In [ ]:
# Create a mask with values of True where the following condition is met, otherwise False.
mask = (ds_merged.river_name == RIVER_NAME).compute()

In [ ]:
# Use the mask to filter our data
ds_filtered = ds_merged.where(mask, drop=True)
ds_filtered

## 5 Locate and analyze discharge timeseries for a reach of interest



We begin by visualizing all reach centerpoints (`x`,`y`) along the river of interest using an interactive map. You can hover over each point to view the reach_id, then identify a specific reach to explore its estimated discharge over time.

In [ ]:
# Convert xarray to pandas DataFrame
df = ds_filtered[['x', 'y', 'reach_id']].to_dataframe().reset_index()

# Rename for clarity
df = df.rename(columns={"x": "lon", "y": "lat"})

# Create interactive scatter plot with Mapbox
fig = px.scatter_map(
    df,
    lat="lat",
    lon="lon",
    hover_name="reach_id",
    zoom=5,
    height=700,
    color_discrete_sequence=["blue"],
    title=f"{RIVER_NAME} Reach Centerpoint Locations"
)

fig.show()


Print the first 10 reach identifiers associated with the selected river. This can help you pick a specific reach of interest to analyze further.

In [ ]:
print(f'{RIVER_NAME} consists of the following reach identifiers')
for rid in ds_filtered.reach_id.values[:10]:
    print(str(rid))
print('...')

To plot the estimated discharge for the reach of interest, we first identify the index of the selected `reach_id` within the dataset. Using this index, we extract the corresponding discharge time series values associated with that specific reach.

In [ ]:
reach_id = 23267000021.0
reach_idx = np.where(ds_filtered.reach_id == reach_id)[0].item()
discharge = ds_filtered.Q.values[reach_idx]

Since the temporal data in the SWOT SoS files is stored as seconds elapsed since January 1, 2000, these values are then converted into standard Python `datetime` objects. This conversion ensures that the discharge data can be correctly plotted on a familiar time scale.

In [ ]:
reference_date = datetime(2000, 1, 1, 0, 0, 0)
times = np.vectorize( lambda t: reference_date + timedelta(seconds=t))( ds_filtered.time.values[reach_idx])

Plot discharge

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=times,
    y=discharge,
    mode='lines+markers',
    name='Discharge',
    line=dict(color='royalblue'),
    marker=dict(size=6)
))

fig.update_layout(
    title=f"Discharge Time Series from HiVDI for the {RIVER_NAME} — Reach ID: {reach_id}",
    xaxis_title='Time',
    yaxis_title='Discharge (m³/s)',
    xaxis_tickformat='%Y-%m-%d',
    height=500
)

fig.show()
